In [10]:
from curriculum import get_exam_curriculum
from utils import *
import random

## Generate questions

In [11]:
bloom_skill_count = 1

In [12]:
question_brd = get_exam_curriculum(
    strand_ids=[1,2,3,4], 
    question_count=4,
    is_debug=True, 
    curriculum_file="data/cbc_data.json",
    bloom_skill_count=bloom_skill_count,
)

✅ Question breakdown written to output/question_breakdown.json


In [13]:
# parsed_output = generate_llm_question_list(
#     llm=OPENAI_LLM_4O,
#     input_data=question_brd,
#     bloom_skill_count=bloom_skill_count,
#     is_debug=True,
# )
parsed_output = [
    {
        "questions": "Amina notices that her phone helps her communicate with friends and learn new things. How does science play a role in making her phone useful in daily life?",
        "expected_answers": "Science contributes to the development of technology like phones, which help in communication, learning, and accessing information."
    },
    {
        "questions": "Brian has a mixture of salt and water. Describe a method he can use to separate the salt from the water.",
        "expected_answers": "Brian can use evaporation to separate the salt from the water. By heating the mixture, the water will evaporate, leaving the salt behind."
    },
    {
        "questions": "Zawadi is learning about the human excretory system. Explain the function of the kidneys in this system.",
        "expected_answers": "The kidneys filter waste products and excess water from the blood to form urine, which is then excreted from the body."
    },
    {
        "questions": "Musa wants to hang a poster on his metal locker using a magnet. Explain why a magnet is suitable for this task.",
        "expected_answers": "A magnet is suitable because it can attract and hold onto the metal surface of the locker, keeping the poster in place without damaging it."
    }
]


In [14]:
QUESTION_LIST_OUTPUT_FILE = "output/question_list.json"

with open(QUESTION_LIST_OUTPUT_FILE, 'w', encoding='utf-8') as f:
    json.dump(parsed_output, f, ensure_ascii=False, indent=4)

print(f"✅ Question list written to {QUESTION_LIST_OUTPUT_FILE}")

✅ Question list written to output/question_list.json


In [15]:
def convert_to_single_qa_list(data):
    result = []
    for index, item in enumerate(data):
        questions = item["questions"]
        expected_answers = item["expected_answers"]

        if isinstance(questions, str):
            questions = [questions]
        if isinstance(expected_answers, str):
            expected_answers = [expected_answers]

        if questions and expected_answers:
            result.append({
                "id": index + 1,
                "question": questions[0],
                "expected_answer": expected_answers[0]
            })

    return result

exam_output = convert_to_single_qa_list(parsed_output)
print(exam_output)

[{'id': 1, 'question': 'Amina notices that her phone helps her communicate with friends and learn new things. How does science play a role in making her phone useful in daily life?', 'expected_answer': 'Science contributes to the development of technology like phones, which help in communication, learning, and accessing information.'}, {'id': 2, 'question': 'Brian has a mixture of salt and water. Describe a method he can use to separate the salt from the water.', 'expected_answer': 'Brian can use evaporation to separate the salt from the water. By heating the mixture, the water will evaporate, leaving the salt behind.'}, {'id': 3, 'question': 'Zawadi is learning about the human excretory system. Explain the function of the kidneys in this system.', 'expected_answer': 'The kidneys filter waste products and excess water from the blood to form urine, which is then excreted from the body.'}, {'id': 4, 'question': 'Musa wants to hang a poster on his metal locker using a magnet. Explain why 

## Simulate student answers

In [16]:
def generate_random_scores(count):
    return [{"id": i+1, "avg_score": random.randint(0, 100)} for i in range(count)]

# sample_students = generate_random_scores(4)
sample_students = [{'id': 1, 'avg_score': 12}, {'id': 2, 'avg_score': 95}, {'id': 3, 'avg_score': 54}, {'id': 4, 'avg_score': 71}]
print(sample_students)

[{'id': 1, 'avg_score': 12}, {'id': 2, 'avg_score': 95}, {'id': 3, 'avg_score': 54}, {'id': 4, 'avg_score': 71}]


In [17]:
# parsed_answers = generate_llm_exam_answers_list(
#     llm=OPENAI_LLM_4O,
#     exam_data=exam_output,
#     student_data=sample_students,
#     is_debug=True,
# )
parsed_answers = [
    {
        "id": 1,
        "answers": [
            {
                "question_id": "1",
                "answer": "Phones are made using science, so they help us talk to friends and learn."
            },
            {
                "question_id": "2",
                "answer": "Maybe he can just let the water dry up and the salt will stay."
            },
            {
                "question_id": "3",
                "answer": "Kidneys clean the blood and make urine."
            },
            {
                "question_id": "4",
                "answer": "Magnets stick to metal, so it can hold the poster."
            }
        ]
    },
    {
        "id": 2,
        "answers": [
            {
                "question_id": "1",
                "answer": "Science is used to create the technology in phones, allowing us to communicate and learn easily."
            },
            {
                "question_id": "2",
                "answer": "Brian can heat the mixture so the water evaporates, leaving the salt behind."
            },
            {
                "question_id": "3",
                "answer": "The kidneys filter out waste and extra water from the blood, creating urine to be excreted."
            },
            {
                "question_id": "4",
                "answer": "A magnet is suitable because it attracts the metal locker, holding the poster without damage."
            }
        ]
    },
    {
        "id": 3,
        "answers": [
            {
                "question_id": "1",
                "answer": "Science helps make phones that we use to talk and learn."
            },
            {
                "question_id": "2",
                "answer": "He can heat the water so it evaporates and leaves the salt."
            },
            {
                "question_id": "3",
                "answer": "Kidneys take out waste from blood and make urine."
            },
            {
                "question_id": "4",
                "answer": "Magnets can stick to metal, so they hold the poster."
            }
        ]
    },
    {
        "id": 4,
        "answers": [
            {
                "question_id": "1",
                "answer": "Science helps in making phones that we use for communication and learning."
            },
            {
                "question_id": "2",
                "answer": "He can use evaporation by heating the mixture to get the salt."
            },
            {
                "question_id": "3",
                "answer": "The kidneys filter waste from the blood and make urine."
            },
            {
                "question_id": "4",
                "answer": "A magnet can hold the poster because it sticks to metal."
            }
        ]
    }
]

In [18]:
ANSWERS_LIST_OUTPUT_FILE = "output/answers_list.json"

with open(ANSWERS_LIST_OUTPUT_FILE, 'w', encoding='utf-8') as f:
    json.dump(parsed_answers, f, ensure_ascii=False, indent=4)

print(f"✅ Answers list written to {ANSWERS_LIST_OUTPUT_FILE}")

✅ Answers list written to output/answers_list.json


## Grade answers